In [20]:
import requests
import PyPDF2
import pandas as pd
import bs4

In [90]:
def getArticleCategoryAndTitle(urlAbs,year,ID):
    abstractHTML = requests.get(url=urlAbs).text
    soup = bs4.BeautifulSoup(abstractHTML, features="xml")
    categoryFrom = abstractHTML.find(f"/abs/{year}.{ID}?context=")+24
    categoryTill = abstractHTML[categoryFrom:].find('"')
    return abstractHTML[categoryFrom:categoryFrom+categoryTill], soup.find("meta", {"name": "citation_title"})['content']

def getArticleTtext(urlPdf, th):
    r = requests.get(url=urlPdf)
    f = open(f"article{th}.pdf","wb")
    f.write(r.content)
    f.close()

    pdfFileObj = open(f"article{th}.pdf", 'rb')
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
    
    articleText = ""

    for i in range(pdfReader.numPages):
        articleText += pdfReader.getPage(i).extract_text()
    
    pdfFileObj.close()
    return articleText

def scrapeArticleText(idsFrom, idsTo, year):
    assert idsFrom > 0, 'idsFrom must be greater than 0'
    df = pd.DataFrame(columns=['year','ID','title','category','text'])

    for i in range(idsFrom, idsTo):
        ID = '%05d' % i
        urlAbs = f"https://arxiv.org/abs/2010.{ID}"
        urlPdf = f"https://arxiv.org/pdf/2010.{ID}.pdf"
        articleCategory, articleTitle = getArticleCategoryAndTitle(urlAbs,year,ID)
        articleText = getArticleTtext(urlPdf, 0)

        df = pd.concat([df, pd.DataFrame({'year':[year],'ID':[ID], 'title':[articleTitle],'category':[articleCategory],'text':[articleText]})])
    return df

def scrapeArticleTitle(idsFrom, idsTo, year):
    assert idsFrom > 0, 'idsFrom must be greater than 0'
    df = pd.DataFrame(columns=['year','ID','title','category','text'])

    for i in range(idsFrom, idsTo):
        ID = '%05d' % i
        urlAbs = f"https://arxiv.org/abs/2010.{ID}"
        articleCategory, articleTitle = getArticleCategoryAndTitle(urlAbs,year,ID)

        df = pd.concat([df, pd.DataFrame({'year':[year],'ID':[ID], 'title':[articleTitle],'category':[articleCategory],'text':[None]})])
    return df

def scrapeSingleTitle(id, year):
    ID = '%05d' % id
    urlAbs = f"https://arxiv.org/abs/2010.{ID}"
    articleCategory, articleTitle = getArticleCategoryAndTitle(urlAbs, year, ID)

    df = pd.DataFrame({'year':[year],'ID':[ID], 'title':[articleTitle],'category':[articleCategory],'text':[None]})
    return df

In [107]:
scrapeArticleText(1, 100, 2010).to_csv('100Articles.csv')

Superfluous whitespace found in object header b'1' b'0'
Superfluous whitespace found in object header b'2' b'0'
Superfluous whitespace found in object header b'3' b'0'
Superfluous whitespace found in object header b'22' b'0'
Superfluous whitespace found in object header b'33' b'0'
Superfluous whitespace found in object header b'36' b'0'
Superfluous whitespace found in object header b'40' b'0'
Superfluous whitespace found in object header b'67' b'0'
Superfluous whitespace found in object header b'70' b'0'
Superfluous whitespace found in object header b'73' b'0'
Superfluous whitespace found in object header b'76' b'0'
Superfluous whitespace found in object header b'79' b'0'
Superfluous whitespace found in object header b'82' b'0'
Superfluous whitespace found in object header b'86' b'0'
Superfluous whitespace found in object header b'91' b'0'
Superfluous whitespace found in object header b'100' b'0'
Superfluous whitespace found in object header b'103' b'0'
Superfluous whitespace found in 

In [104]:
from threading import Thread

class Scrape(Thread):
    def __init__(self):
        Thread.__init__(self)
        # set a default value
        self.df = None
    def run(self, id, year):
        self.df = scrapeSingleTitle(id, year)

thread = Scrape()
thread.start()
thread.run(1, 2010)
thread.run(2, 2010)
print(thread.df)
thread.run(2, 2010)
print(thread.df)

Exception in thread Thread-21:
Traceback (most recent call last):
  File "c:\Users\QbaSo\AppData\Local\Programs\Python\Python39\lib\threading.py", line 973, in _bootstrap_inner
    self.run()
TypeError: run() missing 2 required positional arguments: 'id' and 'year'


   year     ID                                              title  category  \
0  2010  00002  Chemically Homogeneous Evolution: A rapid popu...  astro-ph   

   text  
0  None  
   year     ID                                              title  category  \
0  2010  00002  Chemically Homogeneous Evolution: A rapid popu...  astro-ph   

   text  
0  None  


In [88]:
import threading


t1 = threading.Thread(target=scrapeSingleTitle, args=(1,2010))
t2 = threading.Thread(target=scrapeSingleTitle, args=(2,2010))

# starting thread 1
t1.start()
# starting thread 2
t2.start()

# wait until thread 1 is completely executed
t1.join()
# wait until thread 2 is completely executed
t2.join()

In [89]:
t1.

<Thread(Thread-7, stopped 2864)>